In [1]:
import numpy as np
import pandas as pd
import scipy
from statsmodels.sandbox.stats.multicomp import multipletests 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def write_answer(filename, answer):
    with open(filename, "w") as fout:
        fout.write(str(answer))

In [4]:
genes = pd.read_csv('gene_high_throughput_sequencing.csv')
genes

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,STT5750_Breast_021_DCIS,cancer,1.466240,3.689955,21.331981,19.359546,23.505609,11.576176,11.325851,80.572057,...,2.807635,1.466240,2.807635,1.466240,2.807635,3.689955,18.449777,1.466240,1.466240,1.466240
68,STT5751_Breast_021_IDC,cancer,2.492410,12.279444,30.087759,17.703926,26.064890,10.707081,11.520120,65.041865,...,2.492410,2.492410,1.301619,3.275668,1.301619,1.301619,8.601194,1.301619,1.301619,1.301619
69,STT5758_Breast_022_IDC,cancer,3.530477,20.006038,53.924651,25.449565,30.949995,16.794696,17.121366,70.958462,...,1.176826,1.176826,1.176826,2.961613,1.176826,1.176826,10.764365,1.176826,1.176826,1.176826
70,STT5763_Breast_022_IDC,cancer,3.733734,8.860505,32.538666,21.585069,24.987992,11.460224,11.201202,67.767125,...,2.840946,1.483635,1.483635,1.483635,1.483635,1.483635,8.522837,1.483635,1.483635,1.483635


    Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

    - для групп normal (control) и early neoplasia (treatment)
    - для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [5]:
normal = genes[genes['Diagnosis'] == 'normal'].drop(['Patient_id', 'Diagnosis'], axis=1)
early_neoplasia = genes[genes['Diagnosis'] == 'early neoplasia'].drop(['Patient_id', 'Diagnosis'], axis=1)
cancer = genes[genes['Diagnosis'] == 'cancer'].drop(['Patient_id', 'Diagnosis'], axis=1)

In [6]:
normal.head()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,26.329928,57.553312,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,39.140813,119.572567,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,48.144685,55.579752,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,27.076621,54.454714,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,35.896701,48.134682,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [7]:
def make_ttest_df(control, treatment):
    ttest = []

    for gene in genes.columns[2:]:
        _, p = scipy.stats.ttest_ind(control[gene], treatment[gene], equal_var = False)
        ttest.append([gene, p])
    
    df = pd.DataFrame.from_dict(ttest)
    df.columns = ['gene', 'p']
    df['reject'] = df.p < .05
    
    return df

In [8]:
normal_and_neoplasia = make_ttest_df(normal, early_neoplasia)
neoplasia_and_cancer = make_ttest_df(early_neoplasia, cancer)

In [9]:
normal_and_neoplasia.head()

,gene,p,reject
0,LOC643837,0.690766,False
1,LOC100130417,0.000032,True
2,SAMD11,0.060273,False
3,NOC2L,0.826429,False
4,KLHL17,0.049876,True


In [10]:
neoplasia_and_cancer.head()

,gene,p,reject
0,LOC643837,0.413735,False
1,LOC100130417,0.653429,False
2,SAMD11,0.079556,False
3,NOC2L,0.287581,False
4,KLHL17,0.463292,False


In [11]:
normal_and_neoplasia.reject.sum(), neoplasia_and_cancer.reject.sum()

(1575, 3490)

In [12]:
answer_1_1 = write_answer('answer_1_1.txt', normal_and_neoplasia.reject.sum())
answer_1_2 = write_answer('answer_1_2.txt', neoplasia_and_cancer.reject.sum())

    Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5

In [13]:
def make_multipletests(data, alpha, method):
    reject_corrected, p_corrected, a1, a2 = multipletests(data.p, 
                                                          alpha = alpha,
                                                          method = method) 
    
    df = data
    df['p_' + method] = p_corrected
    df['reject_' + method] = reject_corrected
    
    return df

In [14]:
def fold_change(gene, control, treatment):
    C = control[gene].mean()
    T = treatment[gene].mean()
    return (T/C) if (T>C) else -(C/T)

In [32]:
def make_method_rejected_df(data, method, control, treatment):
    df_method_rejected = data[data['reject_' + method]]
    df_method_rejected['fold_change'] = (df_method_rejected['gene']
                                       .apply(lambda gene: fold_change(gene, control, treatment)))
    df_method_rejected['fc_greater'] = np.absolute(df_method_rejected['fold_change']) > 1.5
    
    return df_method_rejected

In [15]:
normal_and_neoplasia_holm = make_multipletests(normal_and_neoplasia, .05 / 2, 'holm')
neoplasia_and_cancer_holm = make_multipletests(neoplasia_and_cancer, .05 / 2, 'holm')

normal_and_neoplasia_holm.head()

,gene,p,reject,p_holm,reject_holm
0,LOC643837,0.690766,False,1.000000,False
1,LOC100130417,0.000032,True,0.500174,False
2,SAMD11,0.060273,False,1.000000,False
3,NOC2L,0.826429,False,1.000000,False
4,KLHL17,0.049876,True,1.000000,False


In [16]:
neoplasia_and_cancer_holm.head()

,gene,p,reject,p_holm,reject_holm
0,LOC643837,0.413735,False,1.0,False
1,LOC100130417,0.653429,False,1.0,False
2,SAMD11,0.079556,False,1.0,False
3,NOC2L,0.287581,False,1.0,False
4,KLHL17,0.463292,False,1.0,False


In [35]:
normal_and_neoplasia_holm_rejected = make_method_rejected_df(normal_and_neoplasia_holm, 'holm', normal, early_neoplasia)
neoplasia_and_cancer_holm_rejected = make_method_rejected_df(neoplasia_and_cancer_holm, 'holm', early_neoplasia, cancer)

In [36]:
normal_and_neoplasia_holm_rejected.fc_greater.sum(), neoplasia_and_cancer_holm_rejected.fc_greater.sum()

(2, 77)

In [20]:
answer_2_1 = write_answer('answer_2_1.txt', normal_and_neoplasia_holm_rejected.fc_greater.sum())
answer_2_2 = write_answer('answer_2_2.txt', neoplasia_and_cancer_holm_rejected.fc_greater.sum())

    Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0, когда они есть, и будут чаще отклонять H0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5. 

In [21]:
normal_and_neoplasia_bh = make_multipletests(normal_and_neoplasia, .05 / 2, 'fdr_bh')
neoplasia_and_cancer_bh = make_multipletests(neoplasia_and_cancer, .05 / 2, 'fdr_bh')

normal_and_neoplasia_bh.head()

,gene,p,reject,p_holm,reject_holm,p_fdr_bh,reject_fdr_bh
0,LOC643837,0.690766,False,1.000000,False,0.966511,False
1,LOC100130417,0.000032,True,0.500174,False,0.035698,False
2,SAMD11,0.060273,False,1.000000,False,0.536103,False
3,NOC2L,0.826429,False,1.000000,False,0.980777,False
4,KLHL17,0.049876,True,1.000000,False,0.499016,False


In [22]:
neoplasia_and_cancer_bh.head()

,gene,p,reject,p_holm,reject_holm,p_fdr_bh,reject_fdr_bh
0,LOC643837,0.413735,False,1.0,False,0.675195,False
1,LOC100130417,0.653429,False,1.0,False,0.836406,False
2,SAMD11,0.079556,False,1.0,False,0.288873,False
3,NOC2L,0.287581,False,1.0,False,0.563007,False
4,KLHL17,0.463292,False,1.0,False,0.712214,False


In [37]:
normal_and_neoplasia_bh_rejected = make_method_rejected_df(normal_and_neoplasia_bh, 'fdr_bh', normal, early_neoplasia)
neoplasia_and_cancer_bh_rejected = make_method_rejected_df(neoplasia_and_cancer_bh, 'fdr_bh', early_neoplasia, cancer)

In [38]:
normal_and_neoplasia_bh_rejected.head()

,gene,p,reject,p_holm,reject_holm,p_fdr_bh,reject_fdr_bh,fold_change,fc_greater
4627,TMEM63C,2.222228e-06,True,0.034989,False,0.008749,True,1.663826,True
7244,PCSK4,7.955435e-07,True,0.012527,True,0.006264,True,1.509785,True
8250,CACNG8,2.015472e-06,True,0.031736,False,0.008749,True,1.754951,True
9820,EEF1A2,8.498742e-08,True,0.001338,True,0.001338,True,1.974868,True


In [39]:
neoplasia_and_cancer_bh_rejected.head()

,gene,p,reject,p_holm,reject_holm,p_fdr_bh,reject_fdr_bh,fold_change,fc_greater
8,ISG15,7.402400e-04,True,1.000000,False,0.017556,True,2.458867,True
17,FAM132A,2.945987e-04,True,1.000000,False,0.009487,True,-1.329884,False
44,GNB1,1.154858e-03,True,1.000000,False,0.022992,True,1.168026,False
47,GABRD,2.276043e-07,True,0.003575,True,0.000086,True,1.843860,True
54,PLCH2,5.973956e-04,True,1.000000,False,0.015198,True,-1.406503,False


In [40]:
normal_and_neoplasia_bh_rejected.fc_greater.sum(), neoplasia_and_cancer_bh_rejected.fc_greater.sum()

(4, 524)

In [41]:
answer_3_1 = write_answer('answer_3_1.txt', normal_and_neoplasia_bh_rejected.fc_greater.sum())
answer_3_2 = write_answer('answer_3_2.txt', neoplasia_and_cancer_bh_rejected.fc_greater.sum())